# Analyzing NYC High School Data


### <a id='Index'></a> Index
 * [Intro](#Intro)
 * [Import libraries and read data](#Import-libraries-and-read-data)
 * [Cleaning data](#Cleaning-data)
  * [Exploring the SAT Data](#Exploring-the-SAT-Data)
  * [Exploring the Remaining Data](#Exploring-the-Remaining-Data)
  * [Inserting DBN Fields](#Inserting-DBN-Fields)
  * [Combining the SAT Scores](#Combining-the-SAT-Scores)
  * [Parsing Geographic Coordinates for Schools](#Parsing-Geographic-Coordinates-for-Schools)
 * [Combining data](#Combining-data)
  * [Condensing the Class Size Data Set](#Condensing-the-Class-Size-Data-Set)
  * [Computing Average Class Sizes](#Computing-Average-Class-Sizes)
  * [Condensing the Demographics Data Set](#Condensing-the-Demographics-Data-Set)
  * [Condensing the Graduation Data Set](#Condensing-the-Graduation-Data-Set)
  * [Converting AP Test Scores](#Converting-AP-Test-Scores)
  * [Left, Right, Inner, and Outer Joins](#Left,-Right,-Inner,-and-Outer-Joins)
  * [Performing the Left Joins](#Performing-the-Left-Joins)
  * [Performing the Inner Joins](#Performing-the-Inner-Joins)
  * [Filling in Missing Values](#Filling-in-Missing-Values)
 * [Analyzing and Visualizing the Data](#Analyzing-and-Visualizing-the-Data)

---

## Intro
[Back](#Index)

One of the most controversial issues in the U.S. educational system is the efficacy of standardized tests, and whether they're unfair to certain groups. Given our prior knowledge of this topic, investigating the correlations between [SAT](https://en.wikipedia.org/wiki/SAT) scores and demographics might be an interesting angle to take. We could correlate SAT scores with factors like race, gender, income, and more.

The SAT, or Scholastic Aptitude Test, is an exam that U.S. high school students take before applying to college. Colleges take the test scores into account when deciding who to admit, so it's fairly important to perform well on it.

The test consists of three sections, each of which has 800 possible points. The combined score is out of 2,400 possible points (while this number has changed a few times, the data set for our project is based on 2,400 total points). Organizations often rank high schools by their average SAT scores. The scores are also considered a measure of overall school district quality.

New York City makes its [data on high school SAT scores](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4) available online, as well as the [demographics for each high school](https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2).

Unfortunately, combining both of the data sets won't give us all of the demographic information we want to use. We'll need to supplement our data with other sources to do our full analysis.

The same website has several related data sets covering demographic information and test scores. Here are the links to all of the data sets we'll be using:

* [SAT scores by school](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4) - SAT scores for each high school in New York City
* [School attendance](https://data.cityofnewyork.us/Education/School-Attendance-and-Enrollment-Statistics-by-Dis/7z8d-msnt) - Attendance information for each school in New York City
* [Class size](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3) - Information on class size for each school
* [AP test results](https://data.cityofnewyork.us/Education/AP-College-Board-2010-School-Level-Results/itfs-ms3e) - Advanced Placement (AP) exam results for each high school (passing an optional AP exam in a particular subject can earn a student college credit in that subject)
* [Graduation outcomes](https://data.cityofnewyork.us/Education/Graduation-Outcomes-Classes-Of-2005-2010-School-Le/vh2h-md7a) - The percentage of students who graduated, and other outcome information
* [Demographics](https://data.cityofnewyork.us/Education/School-Demographics-and-Accountability-Snapshot-20/ihfw-zy9j) - Demographic information for each school
* [School survey](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8) - Surveys of parents, teachers, and students at each school

All of these data sets are interrelated. We'll need to combine them into a single data set before we can find correlations.

A thorough understanding of the data will help us avoid costly mistakes, such as thinking that a column represents something other than what it does. Background research will also give us a better understanding of how to combine and analyze the data.

In this case, we'll want to research:

* [New York City](https://en.wikipedia.org/wiki/New_York_City)
* [The SAT](https://en.wikipedia.org/wiki/SAT)
* [Schools in New York City](https://en.wikipedia.org/wiki/List_of_high_schools_in_New_York_City)
* [Our data](https://data.cityofnewyork.us/browse?category=Education)

We can learn a few different things from these resources. For example:

* Only high school students take the SAT, so we'll want to focus on high schools.
* New York City is made up of five boroughs, which are essentially distinct regions.
* New York City schools fall within several different school districts, each of which can contains dozens of schools.
* Our data sets include several different types of schools. We'll need to clean them so that we can focus on high schools only.
* Each school in New York City has a unique code called a DBN, or district borough number.
* Aggregating data by district will allow us to use the district mapping data to plot district-by-district differences.

---

We've placed all the data into the schools folder. Here are all of the files in the folder:

* `ap_2010.csv` - Data on [AP test results](https://data.cityofnewyork.us/Education/AP-College-Board-2010-School-Level-Results/itfs-ms3e)
* `class_size.csv` - Data on [class size](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3)
* `demographics.csv` - Data on [demographics](https://data.cityofnewyork.us/Education/School-Demographics-and-Accountability-Snapshot-20/ihfw-zy9j)
* `graduation.csv` - Data on [graduation outcomes](https://data.cityofnewyork.us/Education/Graduation-Outcomes-Classes-Of-2005-2010-School-Le/vh2h-md7a)
* `hs_directory.csv` - A directory of [high schools](https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2)
* `sat_results.csv` - Data on [SAT scores](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4)
* `survey_all.txt` - Data on [surveys](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8) from all schools
* `survey_d75.txt` - Data on [surveys](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8) from New York City [district 75](http://schools.nyc.gov/academics/specialEducation/D75/default.htm)

We'll read each file into a pandas dataframe, and then store all of the dataframes in a dictionary. This will give us a convenient way to store them, and a quick way to reference them later on.

---

## Import libraries and read data
[Back](#Index)

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]
data = {}

for file in data_files:
    key = file.replace('.csv', '')
    data[key] = pd.read_csv('schools/' + file)
    data[key].rename(str.strip, axis='columns', inplace=True) # delete leading and trailing whitespaces (including newlines) from column's names
    
# non csv

all_survey = pd.read_csv('schools/survey_all.txt', encoding='windows-1252', delimiter='\t')
d75_survey = pd.read_csv('schools/survey_d75.txt', encoding='windows-1252', delimiter='\t')

survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

survey['DBN'] = survey['dbn']

# meaningfull columns
survey = survey[["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]]

data['survey'] = survey

---

## Cleaning data
[Back](#Index)

### Exploring the SAT Data
[Back](#Index)

What we're mainly interested in is the SAT data set, which corresponds to the dictionary key `sat_results`. This data set contains the SAT scores for each high school in New York City. We eventually want to correlate selected information from this data set with information in the other data sets.

Let's explore `sat_results` to see what we can discover. Exploring the dataframe will help us understand the structure of the data, and make it easier for us to analyze it.

In [3]:
data['sat_results'].head()

DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   
3                      7                             414                 401   
4                     44                             390                 433   

  SAT Writing Avg. Score  
0                    363  
1                    366  
2                    370  
3                    359  
4                    384

We can make a few observations based on this output:

* The `DBN` appears to be a unique ID for each school.
* We can tell from the first few rows of names that we only have data about high schools.
* There's only a single row for each high school, so each `DBN` is unique in the SAT data.
* We may eventually want to combine the three columns that contain SAT scores -- `SAT Critical Reading Avg. Score`, `SAT Math Avg. Score`, and `SAT Writing Avg. Score` -- into a single column to make the scores easier to analyze.

### Exploring the Remaining Data
[Back](#Index)

In [4]:
print('***************')
print('*** ap_2010 ***')
print('***************')
data['ap_2010'].head()

***************
*** ap_2010 ***
***************


DBN                             SchoolName AP Test Takers  \
0  01M448           UNIVERSITY NEIGHBORHOOD H.S.             39   
1  01M450                 EAST SIDE COMMUNITY HS             19   
2  01M515                    LOWER EASTSIDE PREP             24   
3  01M539         NEW EXPLORATIONS SCI,TECH,MATH            255   
4  02M296  High School of Hospitality Management              s   

  Total Exams Taken Number of Exams with scores 3 4 or 5  
0                49                                   10  
1                21                                    s  
2                26                                   24  
3               377                                  191  
4                 s                                    s

In [5]:
print('******************')
print('*** class_size ***')
print('******************')

data['class_size'].head()

******************
*** class_size ***
******************


CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente    0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente    0K          CTT   
2    1       M        M015  P.S. 015 Roberto Clemente    01       GEN ED   
3    1       M        M015  P.S. 015 Roberto Clemente    01          CTT   
4    1       M        M015  P.S. 015 Roberto Clemente    02       GEN ED   

  CORE SUBJECT (MS CORE and 9-12 ONLY) CORE COURSE (MS CORE and 9-12 ONLY)  \
0                                    -                                   -   
1                                    -                                   -   
2                                    -                                   -   
3                                    -                                   -   
4                                    -                                   -   

  SERVICE CATEGORY(K-9* ONLY)  NUMBER OF STUDENTS / SEATS FILLED  \
0                           -                               19.0   
1                           -                               21.0   
2                           -                               17.0   
3                           -                               17.0   
4                           -                               15.0   

   NUMBER OF SECTIONS  AVERAGE CLASS SIZE  SIZE OF SMALLEST CLASS  \
0                 1.0                19.0                    19.0   
1                 1.0                21.0                    21.0   
2                 1.0                17.0                    17.0   
3                 1.0                17.0                    17.0   
4                 1.0                15.0                    15.0   

   SIZE OF LARGEST CLASS DATA SOURCE  SCHOOLWIDE PUPIL-TEACHER RATIO  
0                   19.0         ATS                             NaN  
1                   21.0         ATS                             NaN  
2                   17.0         ATS                             NaN  
3                   17.0         ATS                             NaN  
4                   15.0         ATS                             NaN

In [6]:
print('********************')
print('*** demographics ***')
print('********************')

data['demographics'].head()

********************
*** demographics ***
********************


DBN                       Name  schoolyear fl_percent  frl_percent  \
0  01M015  P.S. 015 ROBERTO CLEMENTE    20052006       89.4          NaN   
1  01M015  P.S. 015 ROBERTO CLEMENTE    20062007       89.4          NaN   
2  01M015  P.S. 015 ROBERTO CLEMENTE    20072008       89.4          NaN   
3  01M015  P.S. 015 ROBERTO CLEMENTE    20082009       89.4          NaN   
4  01M015  P.S. 015 ROBERTO CLEMENTE    20092010                    96.5   

   total_enrollment prek   k grade1 grade2  ... black_num black_per  \
0               281   15  36     40     33  ...        74      26.3   
1               243   15  29     39     38  ...        68      28.0   
2               261   18  43     39     36  ...        77      29.5   
3               252   17  37     44     32  ...        75      29.8   
4               208   16  40     28     32  ...        67      32.2   

  hispanic_num hispanic_per white_num white_per male_num male_per female_num  \
0          189         67.3         5       1.8    158.0     56.2      123.0   
1          153         63.0         4       1.6    140.0     57.6      103.0   
2          157         60.2         7       2.7    143.0     54.8      118.0   
3          149         59.1         7       2.8    149.0     59.1      103.0   
4          118         56.7         6       2.9    124.0     59.6       84.0   

  female_per  
0       43.8  
1       42.4  
2       45.2  
3       40.9  
4       40.4  

[5 rows x 38 columns]

In [7]:
print('********************')
print('*** hs_directory ***')
print('********************')

data['hs_directory'].head()

********************
*** hs_directory ***
********************


dbn                                        school_name       boro  \
0  17K548                Brooklyn School for Music & Theatre   Brooklyn   
1  09X543                   High School for Violin and Dance      Bronx   
2  09X327        Comprehensive Model School Project M.S. 327      Bronx   
3  02M280     Manhattan Early College School for Advertising  Manhattan   
4  28Q680  Queens Gateway to Health Sciences Secondary Sc...     Queens   

  building_code    phone_number    fax_number grade_span_min  grade_span_max  \
0          K440    718-230-6250  718-230-6262              9              12   
1          X400    718-842-0687  718-589-9849              9              12   
2          X240    718-294-8111  718-294-8109              6              12   
3          M520  718-935-3477             NaN              9              10   
4          Q695    718-969-3155  718-969-3552              6              12   

  expgrade_span_min  expgrade_span_max  ...  \
0               NaN                NaN  ...   
1               NaN                NaN  ...   
2               NaN                NaN  ...   
3                 9               14.0  ...   
4               NaN                NaN  ...   

                                          priority02  \
0                    Then to New York City residents   
1  Then to New York City residents who attend an ...   
2  Then to Bronx students or residents who attend...   
3  Then to New York City residents who attend an ...   
4  Then to Districts 28 and 29 students or residents   

                                          priority03  \
0                                                NaN   
1                Then to Bronx students or residents   
2  Then to New York City residents who attend an ...   
3          Then to Manhattan students or residents     
4               Then to Queens students or residents   

                            priority04                       priority05  \
0                                  NaN                              NaN   
1      Then to New York City residents                              NaN   
2  Then to Bronx students or residents  Then to New York City residents   
3      Then to New York City residents                              NaN   
4      Then to New York City residents                              NaN   

  priority06  priority07 priority08  priority09 priority10  \
0        NaN         NaN        NaN         NaN        NaN   
1        NaN         NaN        NaN         NaN        NaN   
2        NaN         NaN        NaN         NaN        NaN   
3        NaN         NaN        NaN         NaN        NaN   
4        NaN         NaN        NaN         NaN        NaN   

                                          Location 1  
0  883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...  
1  1110 Boston Road\nBronx, NY 10456\n(40.8276026...  
2  1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...  
3  411 Pearl Street\nNew York, NY 10038\n(40.7106...  
4  160-20 Goethals Avenue\nJamaica, NY 11432\n(40...  

[5 rows x 58 columns]

In [8]:
print('**************')
print('*** survey ***')
print('**************')

data['survey'].head()

**************
*** survey ***
**************


DBN  rr_s  rr_t  rr_p    N_s   N_t    N_p  saf_p_11  com_p_11  eng_p_11  \
0  01M015   NaN    88    60    NaN  22.0   90.0       8.5       7.6       7.5   
1  01M019   NaN   100    60    NaN  34.0  161.0       8.4       7.6       7.6   
2  01M020   NaN    88    73    NaN  42.0  367.0       8.9       8.3       8.3   
3  01M034  89.0    73    50  145.0  29.0  151.0       8.8       8.2       8.0   
4  01M063   NaN   100    60    NaN  23.0   90.0       8.7       7.9       8.1   

   ...  eng_t_11  aca_t_11  saf_s_11  com_s_11  eng_s_11  aca_s_11  \
0  ...       7.6       7.9       NaN       NaN       NaN       NaN   
1  ...       8.9       9.1       NaN       NaN       NaN       NaN   
2  ...       6.8       7.5       NaN       NaN       NaN       NaN   
3  ...       6.8       7.8       6.2       5.9       6.5       7.4   
4  ...       7.8       8.1       NaN       NaN       NaN       NaN   

   saf_tot_11  com_tot_11  eng_tot_11  aca_tot_11  
0         8.0         7.7         7.5         7.9  
1         8.5         8.1         8.2         8.4  
2         8.2         7.3         7.5         8.0  
3         7.3         6.7         7.1         7.9  
4         8.5         7.6         7.9         8.0  

[5 rows x 23 columns]

We can make some observations based on the first few rows of each one.

* Each data set appears to either have a DBN column, or the information we need to create one. That means we can use a `DBN` column to combine the data sets. First we'll pinpoint matching rows from different data sets by looking for identical `DBN`s, then group all of their columns together in a single data set.
* Some fields look interesting for mapping -- particularly `Location 1`, which contains coordinates inside a larger string.
* Some of the data sets appear to contain multiple rows for each school (because the rows have duplicate `DBN` values). That means we’ll have to do some preprocessing to ensure that each `DBN` is unique within each data set. If we don't do this, we'll run into problems when we combine the data sets, because we might be merging two rows in one data set with one row in another data set.

###  Inserting DBN Fields
[Back](#Index)

When we explored all of the data sets, we noticed that some of them, like class_size and hs_directory, don't have a `DBN` column. `hs_directory` does have a dbn column, though, so we can just rename it.

From looking at `class_size` rows, we can tell that the DBN in the `sat_results` data is just a combination of the `CSD` and `SCHOOL CODE` columns in the class_size data. The main difference is that the `DBN` is padded, so that the `CSD` portion of it always consists of two digits. That means we'll need to add a leading 0 to the `CSD` if the `CSD` is less than two digits long.

In [9]:
data['hs_directory']['DBN'] = data['hs_directory']['dbn']
data['class_size']['padded_csd'] = data['class_size']['CSD'].apply(lambda x: str(x).zfill(2))
data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']
data['class_size'].head()

CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente    0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente    0K          CTT   
2    1       M        M015  P.S. 015 Roberto Clemente    01       GEN ED   
3    1       M        M015  P.S. 015 Roberto Clemente    01          CTT   
4    1       M        M015  P.S. 015 Roberto Clemente    02       GEN ED   

  CORE SUBJECT (MS CORE and 9-12 ONLY) CORE COURSE (MS CORE and 9-12 ONLY)  \
0                                    -                                   -   
1                                    -                                   -   
2                                    -                                   -   
3                                    -                                   -   
4                                    -                                   -   

  SERVICE CATEGORY(K-9* ONLY)  NUMBER OF STUDENTS / SEATS FILLED  \
0                           -                               19.0   
1                           -                               21.0   
2                           -                               17.0   
3                           -                               17.0   
4                           -                               15.0   

   NUMBER OF SECTIONS  AVERAGE CLASS SIZE  SIZE OF SMALLEST CLASS  \
0                 1.0                19.0                    19.0   
1                 1.0                21.0                    21.0   
2                 1.0                17.0                    17.0   
3                 1.0                17.0                    17.0   
4                 1.0                15.0                    15.0   

   SIZE OF LARGEST CLASS DATA SOURCE  SCHOOLWIDE PUPIL-TEACHER RATIO  \
0                   19.0         ATS                             NaN   
1                   21.0         ATS                             NaN   
2                   17.0         ATS                             NaN   
3                   17.0         ATS                             NaN   
4                   15.0         ATS                             NaN   

  padded_csd     DBN  
0         01  01M015  
1         01  01M015  
2         01  01M015  
3         01  01M015  
4         01  01M015

### <a name="Combining-the-SAT-Scores"></a> Combining the SAT Scores
[Back](#Index)

Now we're almost ready to combine our data sets. Before we do, let's take some time to calculate variables that will be useful in our analysis. We've already discussed one such variable -- a column that totals up the SAT scores for the different sections of the exam. This will make it much easier to correlate scores with demographic factors because we'll be working with a single number, rather than three different ones.

Before we can generate this column, we'll need to convert the `SAT Math Avg. Score`, `SAT Critical Reading Avg. Score`, and `SAT Writing Avg. Score` columns in the `sat_results` data set from the object (string) data type to a numeric data type.

In [10]:
sat_results = data['sat_results']
sat_results['SAT Math Avg. Score'] = pd.to_numeric(sat_results['SAT Math Avg. Score'], errors="coerce")
sat_results['SAT Critical Reading Avg. Score'] = pd.to_numeric(sat_results['SAT Critical Reading Avg. Score'], errors="coerce")
sat_results['SAT Writing Avg. Score'] = pd.to_numeric(sat_results['SAT Writing Avg. Score'], errors="coerce")

sat_results['sat_score'] = sat_results['SAT Math Avg. Score'] + sat_results['SAT Critical Reading Avg. Score'] + sat_results['SAT Writing Avg. Score']

sat_results['sat_score'].head()

0    1122.0
1    1172.0
2    1149.0
3    1174.0
4    1207.0
Name: sat_score, dtype: float64

### <a id="Parsing-Geographic-Coordinates-for-Schools"></a> Parsing Geographic Coordinates for Schools
[Back](#Index)

Next, we'll want to parse the latitude and longitude coordinates for each school. This will enable us to map the schools and uncover any geographic patterns in the data. The coordinates are currently in the text field `Location 1` in the `hs_directory` data set.

In [11]:
import re

def extract_lat(x):
    results = re.search('\((\-?\d+\.\d+)\,\s(\-?\d+\.\d+)', x)
    if results:
        groups = results.groups()
        lat = pd.to_numeric(groups[0], errors="coerce")
        lon = pd.to_numeric(groups[1], errors="coerce")
        return pd.Series([lat, lon])
    return null

hs_directory = data['hs_directory']

hs_directory[['lat', 'lon']] = hs_directory['Location 1'].apply(extract_lat)

hs_directory.head()

dbn                                        school_name       boro  \
0  17K548                Brooklyn School for Music & Theatre   Brooklyn   
1  09X543                   High School for Violin and Dance      Bronx   
2  09X327        Comprehensive Model School Project M.S. 327      Bronx   
3  02M280     Manhattan Early College School for Advertising  Manhattan   
4  28Q680  Queens Gateway to Health Sciences Secondary Sc...     Queens   

  building_code    phone_number    fax_number grade_span_min  grade_span_max  \
0          K440    718-230-6250  718-230-6262              9              12   
1          X400    718-842-0687  718-589-9849              9              12   
2          X240    718-294-8111  718-294-8109              6              12   
3          M520  718-935-3477             NaN              9              10   
4          Q695    718-969-3155  718-969-3552              6              12   

  expgrade_span_min  expgrade_span_max  ...                       priority05  \
0               NaN                NaN  ...                              NaN   
1               NaN                NaN  ...                              NaN   
2               NaN                NaN  ...  Then to New York City residents   
3                 9               14.0  ...                              NaN   
4               NaN                NaN  ...                              NaN   

  priority06 priority07 priority08 priority09  priority10  \
0        NaN        NaN        NaN        NaN         NaN   
1        NaN        NaN        NaN        NaN         NaN   
2        NaN        NaN        NaN        NaN         NaN   
3        NaN        NaN        NaN        NaN         NaN   
4        NaN        NaN        NaN        NaN         NaN   

                                          Location 1     DBN        lat  \
0  883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...  17K548  40.670299   
1  1110 Boston Road\nBronx, NY 10456\n(40.8276026...  09X543  40.827603   
2  1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...  09X327  40.842414   
3  411 Pearl Street\nNew York, NY 10038\n(40.7106...  02M280  40.710679   
4  160-20 Goethals Avenue\nJamaica, NY 11432\n(40...  28Q680  40.718810   

         lon  
0 -73.961648  
1 -73.904475  
2 -73.916162  
3 -74.000807  
4 -73.806500  

[5 rows x 61 columns]

---

## <a name='Combining-data'></a> Combining data
[Back](#Index)

Now we're going to clean the data a bit more, then combine it. Finally, we'll compute correlations and perform some analysis.

The first thing we'll need to do in preparation for the merge is condense some of the data sets. We noticed that the values in the `DBN` column were unique in the `sat_results` data set. **Other data sets like** `class_size` **had duplicate** `DBN` **values, however**.

**We'll need to condense these data sets so that each value in the** `DBN` **column is unique**. If not, we'll run into issues when it comes time to combine the data sets.

While the main data set we want to analyze, `sat_results`, has unique `DBN` values for every high school in New York City, other data sets aren't as clean. A single row in the `sat_results` data set may match multiple rows in the `class_size` data set, for example. This situation will create problems, because we don't know which of the multiple entries in the `class_size` data set we should combine with the single matching entry in `sat_results`.

The first data set that we'll condense is `class_size`. The first few rows of `class_size` look like this:

In [12]:
data['class_size'].head(10)

CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente    0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente    0K          CTT   
2    1       M        M015  P.S. 015 Roberto Clemente    01       GEN ED   
3    1       M        M015  P.S. 015 Roberto Clemente    01          CTT   
4    1       M        M015  P.S. 015 Roberto Clemente    02       GEN ED   
5    1       M        M015  P.S. 015 Roberto Clemente    02          CTT   
6    1       M        M015  P.S. 015 Roberto Clemente    03       GEN ED   
7    1       M        M015  P.S. 015 Roberto Clemente    03          CTT   
8    1       M        M015  P.S. 015 Roberto Clemente    04       GEN ED   
9    1       M        M015  P.S. 015 Roberto Clemente    05       GEN ED   

  CORE SUBJECT (MS CORE and 9-12 ONLY) CORE COURSE (MS CORE and 9-12 ONLY)  \
0                                    -                                   -   
1                                    -                                   -   
2                                    -                                   -   
3                                    -                                   -   
4                                    -                                   -   
5                                    -                                   -   
6                                    -                                   -   
7                                    -                                   -   
8                                    -                                   -   
9                                    -                                   -   

  SERVICE CATEGORY(K-9* ONLY)  NUMBER OF STUDENTS / SEATS FILLED  \
0                           -                               19.0   
1                           -                               21.0   
2                           -                               17.0   
3                           -                               17.0   
4                           -                               15.0   
5                           -                               17.0   
6                           -                               12.0   
7                           -                               15.0   
8                           -                               26.0   
9                           -                               27.0   

   NUMBER OF SECTIONS  AVERAGE CLASS SIZE  SIZE OF SMALLEST CLASS  \
0                 1.0                19.0                    19.0   
1                 1.0                21.0                    21.0   
2                 1.0                17.0                    17.0   
3                 1.0                17.0                    17.0   
4                 1.0                15.0                    15.0   
5                 1.0                17.0                    17.0   
6                 1.0                12.0                    12.0   
7                 1.0                15.0                    15.0   
8                 2.0                13.0                    12.0   
9                 1.0                27.0                    27.0   

   SIZE OF LARGEST CLASS DATA SOURCE  SCHOOLWIDE PUPIL-TEACHER RATIO  \
0                   19.0         ATS                             NaN   
1                   21.0         ATS                             NaN   
2                   17.0         ATS                             NaN   
3                   17.0         ATS                             NaN   
4                   15.0         ATS                             NaN   
5                   17.0         ATS                             NaN   
6                   12.0         ATS                             NaN   
7                   15.0         ATS                             NaN   
8                   14.0         ATS                             NaN   
9                   27.0         ATS                             NaN   

  pad

As you can see, the first few rows all pertain to the same school, which is why the **DBN appears more than once**. It looks like each school has multiple values for `GRADE`, `PROGRAM TYPE`, `CORE SUBJECT (MS CORE and 9-12 ONLY)`, and `CORE COURSE (MS CORE and 9-12 ONLY)`.

If we look at the unique values for GRADE, we get the following:

In [13]:
data['class_size']['GRADE'].unique()

array(['0K', '01', '02', '03', '04', '05', '0K-09', nan, '06', '07', '08',
       'MS Core', '09-12', '09'], dtype=object)

**Because we're dealing with high schools, we're only concerned with grades** `9` **through** `12`. That means we only want to pick rows where the value in the `GRADE` column is `09-12`

If we look at the unique values for `PROGRAM TYPE`, we get the following:

In [14]:
data['class_size']['PROGRAM TYPE'].unique()

array(['GEN ED', 'CTT', 'SPEC ED', nan, 'G&T'], dtype=object)

Each school can have multiple program types. Because `GEN ED` is the largest category by far, let's only select rows where `PROGRAM TYPE` is `GEN ED`

In [15]:
data['class_size']['PROGRAM TYPE'].value_counts()

GEN ED     14545
CTT         7460
SPEC ED     3653
G&T          469
Name: PROGRAM TYPE, dtype: int64

### <a id="Condensing-the-Class-Size-Data-Set"></a> Condensing the Class Size Data Set
[Back](#Index)

In [16]:
data['class_size'] = data['class_size'][
    (data['class_size']['GRADE']=='09-12') & 
    (data['class_size']['PROGRAM TYPE']=='GEN ED')
]
data['class_size'].head()

CSD BOROUGH SCHOOL CODE                                    SCHOOL NAME  \
225    1       M        M292  Henry Street School for International Studies   
226    1       M        M292  Henry Street School for International Studies   
227    1       M        M292  Henry Street School for International Studies   
228    1       M        M292  Henry Street School for International Studies   
229    1       M        M292  Henry Street School for International Studies   

     GRADE PROGRAM TYPE CORE SUBJECT (MS CORE and 9-12 ONLY)  \
225  09-12       GEN ED                              ENGLISH   
226  09-12       GEN ED                              ENGLISH   
227  09-12       GEN ED                              ENGLISH   
228  09-12       GEN ED                              ENGLISH   
229  09-12       GEN ED                                 MATH   

    CORE COURSE (MS CORE and 9-12 ONLY) SERVICE CATEGORY(K-9* ONLY)  \
225                           English 9                           -   
226                          English 10                           -   
227                          English 11                           -   
228                          English 12                           -   
229                  Integrated Algebra                           -   

     NUMBER OF STUDENTS / SEATS FILLED  NUMBER OF SECTIONS  \
225                               63.0                 3.0   
226                               79.0                 3.0   
227                               38.0                 2.0   
228                               69.0                 3.0   
229                               53.0                 3.0   

     AVERAGE CLASS SIZE  SIZE OF SMALLEST CLASS  SIZE OF LARGEST CLASS  \
225                21.0                    19.0                   25.0   
226                26.3                    24.0                   31.0   
227                19.0                    16.0                   22.0   
228                23.0                    13.0                   30.0   
229                17.7                    16.0                   21.0   

    DATA SOURCE  SCHOOLWIDE PUPIL-TEACHER RATIO padded_csd     DBN  
225       STARS                             NaN         01  01M292  
226       STARS                             NaN         01  01M292  
227       STARS                             NaN         01  01M292  
228       STARS                             NaN         01  01M292  
229       STARS                             NaN         01  01M292

### <a id='Computing-Average-Class-Sizes'></a> Computing Average Class Sizes
[Back](#Index)

As we saw when we displayed class_size on the last screen, `DBN` still isn't completely unique. **This is due to the** `CORE COURSE (MS CORE and 9-12 ONLY)` **and** `CORE SUBJECT (MS CORE and 9-12 ONLY)` **columns**.

`CORE COURSE (MS CORE and 9-12 ONLY)` and `CORE SUBJECT (MS CORE and 9-12 ONLY)` seem to pertain to different kinds of classes. For example, here are the unique values for `CORE SUBJECT (MS CORE and 9-12 ONLY)`:

In [17]:
data['class_size']['CORE SUBJECT (MS CORE and 9-12 ONLY)'].unique()

array(['ENGLISH', 'MATH', 'SCIENCE', 'SOCIAL STUDIES'], dtype=object)

This column only seems to include certain subjects. We want our class size data to include every single class a school offers -- not just a subset of them. **What we can do is take the average across all of the classes a school offers. This will give us unique `DBN` values, while also incorporating as much data as possible into the average**.

In [18]:
data['class_size'] = data['class_size'].groupby('DBN').agg(np.mean)
data['class_size'].reset_index(inplace=True)

In [19]:
data['class_size'].head()

DBN  CSD  NUMBER OF STUDENTS / SEATS FILLED  NUMBER OF SECTIONS  \
0  01M292    1                            88.0000            4.000000   
1  01M332    1                            46.0000            2.000000   
2  01M378    1                            33.0000            1.000000   
3  01M448    1                           105.6875            4.750000   
4  01M450    1                            57.6000            2.733333   

   AVERAGE CLASS SIZE  SIZE OF SMALLEST CLASS  SIZE OF LARGEST CLASS  \
0           22.564286                   18.50              26.571429   
1           22.000000                   21.00              23.500000   
2           33.000000                   33.00              33.000000   
3           22.231250                   18.25              27.062500   
4           21.200000                   19.40              22.866667   

   SCHOOLWIDE PUPIL-TEACHER RATIO  
0                             NaN  
1                             NaN  
2                             NaN  
3                             NaN  
4                             NaN

### <a id='Condensing-the-Demographics-Data-Set'></a> Condensing the Demographics Data Set
[Back](#Index)

Now that we've finished condensing `class_size`, let's condense `demographics`. The first few rows look like this:

In [20]:
data['demographics'].head()

DBN                       Name  schoolyear fl_percent  frl_percent  \
0  01M015  P.S. 015 ROBERTO CLEMENTE    20052006       89.4          NaN   
1  01M015  P.S. 015 ROBERTO CLEMENTE    20062007       89.4          NaN   
2  01M015  P.S. 015 ROBERTO CLEMENTE    20072008       89.4          NaN   
3  01M015  P.S. 015 ROBERTO CLEMENTE    20082009       89.4          NaN   
4  01M015  P.S. 015 ROBERTO CLEMENTE    20092010                    96.5   

   total_enrollment prek   k grade1 grade2  ... black_num black_per  \
0               281   15  36     40     33  ...        74      26.3   
1               243   15  29     39     38  ...        68      28.0   
2               261   18  43     39     36  ...        77      29.5   
3               252   17  37     44     32  ...        75      29.8   
4               208   16  40     28     32  ...        67      32.2   

  hispanic_num hispanic_per white_num white_per male_num male_per female_num  \
0          189         67.3         5       1.8    158.0     56.2      123.0   
1          153         63.0         4       1.6    140.0     57.6      103.0   
2          157         60.2         7       2.7    143.0     54.8      118.0   
3          149         59.1         7       2.8    149.0     59.1      103.0   
4          118         56.7         6       2.9    124.0     59.6       84.0   

  female_per  
0       43.8  
1       42.4  
2       45.2  
3       40.9  
4       40.4  

[5 rows x 38 columns]

In this case, the only column that prevents a given `DBN` from being unique is `schoolyear`. We only want to select rows where `schoolyear` is `20112012`. **This will give us the most recent year of data, and also match our** `SAT` **results data**.

In [21]:
data['demographics'] = data['demographics'][data['demographics']['schoolyear'] == 20112012]
data['demographics'].head()

DBN                                              Name  schoolyear  \
6   01M015  P.S. 015 ROBERTO CLEMENTE                           20112012   
13  01M019  P.S. 019 ASHER LEVY                                 20112012   
20  01M020  PS 020 ANNA SILVER                                  20112012   
27  01M034  PS 034 FRANKLIN D ROOSEVELT                         20112012   
35  01M063  PS 063 WILLIAM MCKINLEY                             20112012   

   fl_percent  frl_percent  total_enrollment prek    k grade1 grade2  ...  \
6         NaN         89.4               189   13   31     35     28  ...   
13        NaN         61.5               328   32   46     52     54  ...   
20        NaN         92.5               626   52  102    121     87  ...   
27        NaN         99.7               401   14   34     38     36  ...   
35        NaN         78.9               176   18   20     30     21  ...   

   black_num black_per hispanic_num hispanic_per white_num white_per male_num  \
6         63      33.3          109         57.7         4       2.1     97.0   
13        81      24.7          158         48.2        28       8.5    147.0   
20        55       8.8          357         57.0        16       2.6    330.0   
27        90      22.4          275         68.6         8       2.0    204.0   
35        41      23.3          110         62.5        15       8.5     97.0   

   male_per female_num female_per  
6      51.3       92.0       48.7  
13     44.8      181.0       55.2  
20     52.7      296.0       47.3  
27     50.9      197.0       49.1  
35     55.1       79.0       44.9  

[5 rows x 38 columns]

### <a id='Condensing-the-Graduation-Data-Set'></a> Condensing the Graduation Data Set
[Back](#Index)

Finally, we'll need to condense the `graduation` data set. Here are the first few rows:

In [22]:
data['graduation'].head()

Demographic     DBN                            School Name    Cohort  \
0  Total Cohort  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL      2003   
1  Total Cohort  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL      2004   
2  Total Cohort  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL      2005   
3  Total Cohort  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL      2006   
4  Total Cohort  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL  2006 Aug   

   Total Cohort Total Grads - n Total Grads - % of cohort Total Regents - n  \
0             5               s                         s                 s   
1            55              37                     67.3%                17   
2            64              43                     67.2%                27   
3            78              43                     55.1%                36   
4            78              44                     56.4%                37   

  Total Regents - % of cohort Total Regents - % of grads  ...  \
0                           s                          s  ...   
1                       30.9%                      45.9%  ...   
2                       42.2%                      62.8%  ...   
3                       46.2%                      83.7%  ...   
4                       47.4%                      84.1%  ...   

  Regents w/o Advanced - n Regents w/o Advanced - % of cohort  \
0                        s                                  s   
1                       17                              30.9%   
2                       27                              42.2%   
3                       36                              46.2%   
4                       37                              47.4%   

  Regents w/o Advanced - % of grads Local - n Local - % of cohort  \
0                                 s         s                   s   
1                             45.9%        20               36.4%   
2                             62.8%        16                 25%   
3                             83.7%         7                  9%   
4                             84.1%         7                  9%   

    Local - % of grads Still Enrolled - n Still Enrolled - % of cohort  \
0                    s                  s                            s   
1                54.1%                 15                        27.3%   
2  37.200000000000003%                  9                        14.1%   
3                16.3%                 16                        20.5%   
4                15.9%                 15                        19.2%   

  Dropped Out - n Dropped Out - % of cohort  
0               s                         s  
1               3                      5.5%  
2               9                     14.1%  
3              11                     14.1%  
4              11                     14.1%  

[5 rows x 23 columns]

The `Demographic` and `Cohort` columns are what prevent `DBN` from being unique in the graduation data. A `Cohort` appears to refer to the year the data represents, and the `Demographic` appears to refer to a specific demographic group. **In this case, we want to pick data from the most recent** `Cohort` **available, which is 2006. We also want data from the full cohort, so we'll only pick rows where** `Demographic` **is** `Total Cohort`.

In [23]:
data['graduation'] = data['graduation'][
    (data['graduation']['Demographic'] == 'Total Cohort') & 
    (data['graduation']['Cohort'] == '2006')
]
data['graduation'].head()

Demographic     DBN                            School Name Cohort  \
3   Total Cohort  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL   2006   
10  Total Cohort  01M448    UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   2006   
17  Total Cohort  01M450             EAST SIDE COMMUNITY SCHOOL   2006   
24  Total Cohort  01M509                MARTA VALLE HIGH SCHOOL   2006   
31  Total Cohort  01M515  LOWER EAST SIDE PREPARATORY HIGH SCHO   2006   

    Total Cohort Total Grads - n Total Grads - % of cohort Total Regents - n  \
3             78              43                     55.1%                36   
10           124              53                     42.7%                42   
17            90              70                     77.8%                67   
24            84              47                       56%                40   
31           193             105                     54.4%                91   

   Total Regents - % of cohort Total Regents - % of grads  ...  \
3                        46.2%                      83.7%  ...   
10                       33.9%                      79.2%  ...   
17         74.400000000000006%                      95.7%  ...   
24                       47.6%                      85.1%  ...   
31                       47.2%                      86.7%  ...   

   Regents w/o Advanced - n Regents w/o Advanced - % of cohort  \
3                        36                              46.2%   
10                       34                              27.4%   
17                       67                74.400000000000006%   
24                       23                              27.4%   
31                       22                              11.4%   

   Regents w/o Advanced - % of grads Local - n Local - % of cohort  \
3                              83.7%         7                  9%   
10                             64.2%        11                8.9%   
17                             95.7%         3                3.3%   
24                             48.9%         7  8.300000000000001%   
31                               21%        14                7.3%   

   Local - % of grads Still Enrolled - n Still Enrolled - % of cohort  \
3               16.3%                 16                        20.5%   
10              20.8%                 46                        37.1%   
17               4.3%                 15                        16.7%   
24              14.9%                 25                        29.8%   
31              13.3%                 53                        27.5%   

   Dropped Out - n Dropped Out - % of cohort  
3               11                     14.1%  
10              20       16.100000000000001%  
17               5                      5.6%  
24               5                        6%  
31              35       18.100000000000001%  

[5 rows x 23 columns]

### <a id='Converting-AP-Test-Scores'></a> Converting AP Test Scores
[Back](#Index)

We're almost ready to combine all of the data sets. The only remaining thing to do is convert the [Advanced Placement](https://en.wikipedia.org/wiki/Advanced_Placement_exams) (AP) test scores from strings to numeric values. High school students take the AP exams before applying to college. There are several AP exams, each corresponding to a school subject. High school students who earn high scores may receive college credit.

**AP exams have a 1 to 5 scale; 3 or higher is a passing score**. Many high school students take AP exams -- particularly those who attend academically challenging institutions. AP exams are much more rare in schools that lack funding or academic rigor.

**It will be interesting to find out whether AP exam scores are correlated with SAT scores across high schools**. To determine this, we'll need to convert the AP exam scores in the ap_2010 data set to numeric values first.

There are three columns we'll need to convert:

 - `AP Test Takers`
 - `Total Exams Taken`
 - `Number of Exams with scores 3 4 or 5`

In [27]:
cols = ['AP Test Takers', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for col in cols:
    data['ap_2010'][col] = pd.to_numeric(data['ap_2010'][col], errors='coerce')
    
data['ap_2010'].dtypes

DBN                                      object
SchoolName                               object
AP Test Takers                          float64
Total Exams Taken                       float64
Number of Exams with scores 3 4 or 5    float64
dtype: object

### <a id='Left,-Right,-Inner,-and-Outer-Joins'></a> Left, Right, Inner, and Outer Joins
[Back](#Index)

Before we merge our data, we'll need to decide on the merge strategy we want to use. 
Each merge strategy has its advantages. Depending on the strategy we choose, we may preserve rows at the expense of having more missing column data, or minimize missing data at the expense of having fewer rows. Choosing a merge strategy is an important decision; it's worth thinking about your data carefully, and what trade-offs you're willing to make.

Because this project is concerned with determing demographic factors that correlate with SAT score, we'll want to preserve as many rows as possible from `sat_results` while minimizing null values.

This means that we may need to use different merge strategies with different data sets. **Some of the data sets have a lot of missing** `DBN` **values. This makes a left join more appropriate, because we don't want to lose too many rows when we merge. If we did an inner join, we would lose the data for many high schools**.

Some data sets have `DBN` values that are almost identical to those in `sat_results`. Those data sets also have information we need to keep. Most of our analysis would be impossible if a significant number of rows was missing from `demographics`, for example. Therefore, **we'll do an inner join to avoid missing data in these columns**.

### <a id='Performing-the-Left-Joins'></a> Performing the Left Joins
[Back](#Index)

Both the `ap_2010` and the graduation data sets have many missing `DBN` values, **so we'll use a** `left join` **when we merge the** `sat_results` **data set with them**. Because we're using a `left join`, our final dataframe will have all of the same DBN values as the original `sat_results` dataframe.

In [28]:
combined = data['sat_results'].merge(data['ap_2010'], how='left', on='DBN')
combined = combined.merge(data['graduation'], how='left', on='DBN')

In [29]:
combined.shape

(479, 33)

In [30]:
combined.head()

DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

  Num of SAT Test Takers  SAT Critical Reading Avg. Score  \
0                     29                            355.0   
1                     91                            383.0   
2                     70                            377.0   
3                      7                            414.0   
4                     44                            390.0   

   SAT Math Avg. Score  SAT Writing Avg. Score  sat_score  \
0                404.0                   363.0     1122.0   
1                423.0                   366.0     1172.0   
2                402.0                   370.0     1149.0   
3                401.0                   359.0     1174.0   
4                433.0                   384.0     1207.0   

                     SchoolName  AP Test Takers  Total Exams Taken  ...  \
0                           NaN             NaN                NaN  ...   
1  UNIVERSITY NEIGHBORHOOD H.S.            39.0               49.0  ...   
2        EAST SIDE COMMUNITY HS            19.0               21.0  ...   
3                           NaN             NaN                NaN  ...   
4                           NaN             NaN                NaN  ...   

   Regents w/o Advanced - n Regents w/o Advanced - % of cohort  \
0                        36                              46.2%   
1                        34                              27.4%   
2                        67                74.400000000000006%   
3                       NaN                                NaN   
4                        23                              27.4%   

  Regents w/o Advanced - % of grads Local - n  Local - % of cohort  \
0                             83.7%         7                   9%   
1                             64.2%        11                 8.9%   
2                             95.7%         3                 3.3%   
3                               NaN       NaN                  NaN   
4                             48.9%         7   8.300000000000001%   

  Local - % of grads Still Enrolled - n Still Enrolled - % of cohort  \
0              16.3%                 16                        20.5%   
1              20.8%                 46                        37.1%   
2               4.3%                 15                        16.7%   
3                NaN                NaN                          NaN   
4              14.9%                 25                        29.8%   

  Dropped Out - n Dropped Out - % of cohort  
0              11                     14.1%  
1              20       16.100000000000001%  
2               5                      5.6%  
3             NaN                       NaN  
4               5                        6%  

[5 rows x 33 columns]

### <a id='Performing-the-Inner-Joins'></a> Performing the Inner Joins
[Back](#Index)

Now that we've performed the `left joins`, we still have to merge `class_size`, `demographics`, `survey`, and `hs_directory` into `combined`. **Because these files contain information that's more valuable to our analysis and also have fewer missing** `DBN` **values, we'll use the inner join type**.

In [31]:
combined = combined.merge(data['class_size'], how='inner', on='DBN')
combined = combined.merge(data['demographics'], how='inner', on='DBN')
combined = combined.merge(data['survey'], how='inner', on='DBN')
combined = combined.merge(data['hs_directory'], how='inner', on='DBN')

In [32]:
combined.shape

(363, 159)

In [33]:
combined.head()

DBN                                        SCHOOL NAME  \
0  01M292      HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448                UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                         EAST SIDE COMMUNITY SCHOOL   
3  01M509                            MARTA VALLE HIGH SCHOOL   
4  01M539  NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...   

  Num of SAT Test Takers  SAT Critical Reading Avg. Score  \
0                     29                            355.0   
1                     91                            383.0   
2                     70                            377.0   
3                     44                            390.0   
4                    159                            522.0   

   SAT Math Avg. Score  SAT Writing Avg. Score  sat_score  \
0                404.0                   363.0     1122.0   
1                423.0                   366.0     1172.0   
2                402.0                   370.0     1149.0   
3                433.0                   384.0     1207.0   
4                574.0                   525.0     1621.0   

                       SchoolName  AP Test Takers  Total Exams Taken  ...  \
0                             NaN             NaN                NaN  ...   
1    UNIVERSITY NEIGHBORHOOD H.S.            39.0               49.0  ...   
2          EAST SIDE COMMUNITY HS            19.0               21.0  ...   
3                             NaN             NaN                NaN  ...   
4  NEW EXPLORATIONS SCI,TECH,MATH           255.0              377.0  ...   

                                priority04                       priority05  \
0  Then to Manhattan students or residents  Then to New York City residents   
1                                      NaN                              NaN   
2                                      NaN                              NaN   
3                                      NaN                              NaN   
4                                      NaN                              NaN   

  priority06 priority07  priority08 priority09 priority10  \
0        NaN        NaN         NaN        NaN        NaN   
1        NaN        NaN         NaN        NaN        NaN   
2        NaN        NaN         NaN        NaN        NaN   
3        NaN        NaN         NaN        NaN        NaN   
4        NaN        NaN         NaN        NaN        NaN   

                                          Location 1        lat        lon  
0  220 Henry Street\nNew York, NY 10002\n(40.7137...  40.713764 -73.985260  
1  200 Monroe Street\nNew York, NY 10002\n(40.712...  40.712332 -73.984797  
2  420 East 12 Street\nNew York, NY 10009\n(40.72...  40.729783 -73.983041  
3  145 Stanton Street\nNew York, NY 10002\n(40.72...  40.720569 -73.985673  
4  111 Columbia Street\nNew York, NY 10002\n(40.7...  40.718725 -73.979426  

[5 rows x 159 columns]

### <a id='Filling-in-Missing-Values'></a> Filling in Missing Values
[Back](#Index)

You may have noticed that we now have many columns with null (`NaN`) values. This is because we chose to do `left joins`, where some columns may not have had data. The data set also had some missing values to begin with. **If we hadn't performed a** `left join`, **all of the rows with missing data would have been lost in the merge process, which wouldn't have left us with many high schools in our data set**.

There are several ways to handle missing data, and we'll cover them in more detail later on. For now, **we'll just fill in the missing values with the overall mean for the column**.

In [35]:
means = combined.mean()
combined = combined.fillna(means) # Fill in any missing values in combined with the means
combined = combined.fillna(0) # Fill in any remaining missing values in combined with 0

combined.head()

DBN                                        SCHOOL NAME  \
0  01M292      HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448                UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                         EAST SIDE COMMUNITY SCHOOL   
3  01M509                            MARTA VALLE HIGH SCHOOL   
4  01M539  NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...   

  Num of SAT Test Takers  SAT Critical Reading Avg. Score  \
0                     29                            355.0   
1                     91                            383.0   
2                     70                            377.0   
3                     44                            390.0   
4                    159                            522.0   

   SAT Math Avg. Score  SAT Writing Avg. Score  sat_score  \
0                404.0                   363.0     1122.0   
1                423.0                   366.0     1172.0   
2                402.0                   370.0     1149.0   
3                433.0                   384.0     1207.0   
4                574.0                   525.0     1621.0   

                       SchoolName  AP Test Takers  Total Exams Taken  ...  \
0                               0      129.028846         197.038462  ...   
1    UNIVERSITY NEIGHBORHOOD H.S.       39.000000          49.000000  ...   
2          EAST SIDE COMMUNITY HS       19.000000          21.000000  ...   
3                               0      129.028846         197.038462  ...   
4  NEW EXPLORATIONS SCI,TECH,MATH      255.000000         377.000000  ...   

                                priority04                       priority05  \
0  Then to Manhattan students or residents  Then to New York City residents   
1                                        0                                0   
2                                        0                                0   
3                                        0                                0   
4                                        0                                0   

  priority06 priority07  priority08 priority09 priority10  \
0          0          0           0          0          0   
1          0          0           0          0          0   
2          0          0           0          0          0   
3          0          0           0          0          0   
4          0          0           0          0          0   

                                          Location 1        lat        lon  
0  220 Henry Street\nNew York, NY 10002\n(40.7137...  40.713764 -73.985260  
1  200 Monroe Street\nNew York, NY 10002\n(40.712...  40.712332 -73.984797  
2  420 East 12 Street\nNew York, NY 10009\n(40.72...  40.729783 -73.983041  
3  145 Stanton Street\nNew York, NY 10002\n(40.72...  40.720569 -73.985673  
4  111 Columbia Street\nNew York, NY 10002\n(40.7...  40.718725 -73.979426  

[5 rows x 159 columns]

### <a href="Adding-a-School-District-Column-for-Mapping"></a> Adding a School District Column for Mapping
[Back](#Index)

We've finished cleaning and combining our data! We now have a clean data set on which we can base our analysis. Mapping the statistics out on a school district level might be an interesting way to analyze them. Adding a column to the data set that specifies the school district will help us accomplish this.

The school district is just the first two characters of the `DBN`. We can apply a function over the DBN column of combined that pulls out the first two letters.

In [36]:
combined['school_dist'] = combined['DBN'].apply(lambda x: x[0:2])

combined['school_dist']

0      01
1      01
2      01
3      01
4      01
       ..
358    32
359    32
360    32
361    32
362    32
Name: school_dist, Length: 363, dtype: object

In [ ]:
## Analyzing and Visualizing the Data